# Exploration of geographically weighted random forest classification modelling

To-do:
- [x] global model
- [x] model evaluation
- [x] bandwidth optimisation
- [x] feature importances
- [x] golden section bandwidth selection
- [x] other metrics than accuracy
- [x] generic support (logistic regression, gradient boosting)
- [ ] local performance of models that do not support OOB

In [ ]:
import inspect
import pandas as pd
import numpy as np
import warnings
import geopandas as gpd
from libpysal import graph
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from geodatasets import get_path
from joblib import Parallel, delayed
from typing import Hashable, Callable
from scipy.spatial.distance import pdist
from sklearn import metrics, preprocessing

Get sample data

In [ ]:
gdf = gpd.read_file(get_path("geoda.ncovr"))

In [ ]:
# It is in the geographic coords in the  US and we need to work with distances. Re-project and use only points as the graph builder will require points anyway.
gdf = gdf.set_geometry(gdf.representative_point()).to_crs(5070)

Define a base class for the heavy lifting.

In [ ]:
def _triangular(distances, bandwidth):
    u = np.clip(distances / bandwidth, 0, 1)
    return 1 - u


def _parabolic(distances, bandwidth):
    u = np.clip(distances / bandwidth, 0, 1)
    return 0.75 * (1 - u**2)


def _gaussian(distances, bandwidth):
    u = distances / bandwidth
    return np.exp(-((u / 2) ** 2)) / (np.sqrt(2) * np.pi)


def _bisquare(distances, bandwidth):
    u = np.clip(distances / bandwidth, 0, 1)
    return (15 / 16) * (1 - u**2) ** 2


def _cosine(distances, bandwidth):
    u = np.clip(distances / bandwidth, 0, 1)
    return (np.pi / 4) * np.cos(np.pi / 2 * u)


def _exponential(distances, bandwidth):
    u = distances / bandwidth
    return np.exp(-u)


def _boxcar(distances, bandwidth):
    r = (distances < bandwidth).astype(int)
    return r


_kernel_functions = {
    "triangular": _triangular,
    "parabolic": _parabolic,
    "gaussian": _gaussian,
    "bisquare": _bisquare,
    "cosine": _cosine,
    "boxcar": _boxcar,
    "exponential": _exponential,
}

Note: Classification only at the moment due to hard-coded accuracy.

In [ ]:
class GWM:
    """Generic geographically weighted modelling meta-class

    NOTE: local models leave out focal, unlike in traditional approaches. This allows
    assessment of geographically weighted metrics on unseen data without a need for
    train/test split, hence providing value for all samples. This is needed for
    futher spatial analysis of the model performance (and generalises to models
    that do not support OOB scoring).

    Parameters
    ----------
    model :  model class
        Scikit-learn model class
    bandwidth : int | float
        bandwidth value consisting of either a distance or N nearest neighbors
    fixed : bool, optional
        True for distance based bandwidth and False for adaptive (nearest neighbor) bandwidth, by default False
    kernel : str, optional
        type of kernel function used to weight observations, by default "bisquare"
    n_jobs : int, optional
        The number of jobs to run in parallel. ``-1`` means using all processors by default ``-1``
    fit_global_model : bool, optional
        Determines if the global baseline model shall be fitted alognside the geographically weighted, by default True
    strict : bool, optional
        Do not fit any models if at least one neighborhood has invariant ``y``, by default False
    keep_models : bool, optional
        Keep all local models (required for prediction), by default True. Note that for some models,
        like random forests, the objects can be large.
    **kwargs
        Additional keyword arguments passed to ``model`` initialisation
    """

    def __init__(
        self,
        model,
        bandwidth: int | float,
        fixed: bool = False,
        kernel: str | Callable = "bisquare",
        n_jobs: int = -1,
        fit_global_model: bool = True,
        measure_performance: bool = True,
        strict: bool = False,
        keep_models: bool = False,
        **kwargs,
    ):
        self.model = model
        self.bandwidth = bandwidth
        self.kernel = kernel
        self.fixed = fixed
        self.model_kwargs = kwargs
        self.n_jobs = n_jobs
        self.fit_global_model = fit_global_model
        self.measure_performance = measure_performance
        self.strict = strict
        self.keep_models = keep_models
        self._measure_oob = "oob_score" in inspect.signature(model).parameters
        if self._measure_oob:
            self.model_kwargs["oob_score"] = self._accuracy_data

    def fit(self, X: pd.DataFrame, y: pd.Series, geometry: gpd.GeoSeries):
        """Fit the geographically weighted model

        Parameters
        ----------
        X : pd.DataFrame
            Independent variables
        y : pd.Series
            Dependent variable
        geometry : gpd.GeoSeries
            Geographic location
        """
        # build graph
        if self.fixed:  # fixed distance
            self.weights = graph.Graph.build_kernel(
                geometry, kernel=self.kernel, bandwidth=self.bandwidth
            )
        else:  # adaptive KNN
            weights = graph.Graph.build_kernel(
                geometry, kernel="identity", k=self.bandwidth
            )
            # post-process identity weights by the selected kernel
            # and kernel bandwidth derived from each neighborhood
            bandwidth = weights._adjacency.groupby(level=0).transform("max")
            self.weights = graph.Graph(
                adjacency=_kernel_functions[self.kernel](weights._adjacency, bandwidth),
                is_sorted=True,
            )

        # fit the models
        data = X.copy()
        data["_y"] = y
        data = data.loc[self.weights._adjacency.index.get_level_values(1)]
        data["_weight"] = self.weights._adjacency.values
        grouper = data.groupby(self.weights._adjacency.index.get_level_values(0))

        if self.strict:
            invariant = (
                data["_y"]
                .groupby(self.weights._adjacency.index.get_level_values(0))
                .nunique()
                == 1
            )
            if invariant.any():
                raise ValueError(
                    f"y at locations {invariant.index[invariant]} is invariant."
                )

        # models are fit in parallel
        traning_output = Parallel(n_jobs=self.n_jobs)(
            delayed(self._fit_local)(
                self.model, group, name, focal_x, self.model_kwargs, self.keep_models
            )
            for (name, group), focal_x in zip(grouper, X.values)
        )
        if self.keep_models:
            names, oob_accuracy_data, feature_importances, focal_proba, models = zip(
                *traning_output
            )
            self.local_models = pd.Series(models, index=names)
        else:
            names, oob_accuracy_data, feature_importances, focal_proba = zip(
                *traning_output
            )

        self.focal_proba_ = pd.DataFrame(focal_proba).fillna(0).loc[:, np.unique(y)]

        if self.fit_global_model:
            if self._measure_oob:
                self.model_kwargs["oob_score"] = True
            # fit global model as a baseline
            self.global_model = self.model(n_jobs=self.n_jobs, **self.model_kwargs)
            self.global_model.fit(X=X, y=y)

        if self.measure_performance:
            # global GW accuracy
            focal_pred = self.focal_proba_.idxmax(axis=1)
            self.score_ = metrics.accuracy_score(y, focal_pred)
            self.f1_macro = metrics.f1_score(y, focal_pred, average="macro")
            self.f1_micro = metrics.f1_score(y, focal_pred, average="micro")
            self.f1_weighted = metrics.f1_score(y, focal_pred, average="weighted")

            # OOB accuracy (if model allows)
            if self._measure_oob:
                true, n = zip(*oob_accuracy_data)
                self.oob_score_ = sum(true) / sum(n)
                self.local_oob_score_ = pd.Series(
                    np.array(true) / np.array(n), index=names
                )

        # feature importances (only if supported by the model)
        if feature_importances[0]:
            self.feature_importances_ = pd.DataFrame(
                feature_importances, index=names, columns=X.columns
            )

        return self

    def _fit_local(
        self,
        model,
        data: pd.DataFrame,
        name: Hashable,
        focal_x,
        model_kwargs: dict,
        keep_models: bool,
    ) -> tuple:
        """Fit individual local model

        Parameters
        ----------
        model : model class
            Scikit-learn model class
        data : pd.DataFrame
            data for training
        name : Hashable
            group name, matching the index of the focal geometry
        model_kwargs : dict
            additional keyword arguments for the model init

        Returns
        -------
        tuple
            name, fitted model
        """
        if data["_y"].nunique() == 1:
            warnings.warn(f"y at location {name} is invariant.")
        local_model = model(**model_kwargs)
        local_model.fit(
            X=data.drop(columns=["_y", "_weight"]),
            y=data["_y"],
            sample_weight=data["_weight"],
        )
        focal_x = pd.DataFrame(
            focal_x.reshape(1, -1),
            columns=data.columns.drop(["_y", "_weight"]),
            index=[name],
        )
        focal_proba = pd.Series(
            local_model.predict_proba(focal_x).flatten(), index=local_model.classes_
        )
        output = [
            name,
            local_model.oob_score_ if self._measure_oob else None,
            getattr(local_model, "feature_importances_", None),
            focal_proba,
        ]
        if keep_models:
            output.append(local_model)

        return output

    def _accuracy_data(self, true, pred):
        return sum(true.flatten() == pred), len(pred)

Try with RF Classifier

In [ ]:
gwrf = GWM(
    RandomForestClassifier, bandwidth=250, fixed=False, n_jobs=-1, keep_models=False
)
gwrf.fit(
    gdf.iloc[:, 9:15],
    gdf["STATE_NAME"],
    gdf.geometry,
)

Global OOB score (accuracy) for the GW model measured based on OOB predictions from individual local trees.

In [ ]:
gwrf.oob_score_

Local OOB score.

In [ ]:
gdf.plot(gwrf.local_oob_score_, legend=True, s=2)

Global score (accuracy) for the GW model measured based on prediction of focals.

In [ ]:
gwrf.score_

F1 scores for the GW model measured based on prediction of focals. 

In [ ]:
gwrf.f1_macro, gwrf.f1_micro, gwrf.f1_weighted

OOB score of the global model.

In [ ]:
gwrf.global_model.oob_score_

Get local feature importances.

In [ ]:
gwrf.feature_importances_

In [ ]:
gdf.plot(gwrf.feature_importances_["HC60"], legend=True, s=2)

Compare to global feature importance.

In [ ]:
gwrf.global_model.feature_importances_

Try Logistic regression.

In [ ]:
gwlr = GWM(
    LogisticRegression,
    bandwidth=250,
    fixed=False,
    n_jobs=-1,
    keep_models=False,
    max_iter=500,
)
gwlr.fit(
    pd.DataFrame(
        preprocessing.scale(gdf.iloc[:, 9:15]), columns=gdf.iloc[:, 9:15].columns
    ),
    gdf["STATE_NAME"],
    gdf.geometry,
)

In [ ]:
gwlr.score_

In [ ]:
gwlr.f1_macro, gwlr.f1_micro, gwlr.f1_weighted

Define bandwidth search

In [ ]:
class BandwidthSearch:
    """Optimal bandwidth search for geographically-weighted models

    Minimises one of AIC, AICc, BIC based on prediction probability on focal geometries.

    Parameters
    ----------
    model :  model class
        Scikit-learn model class
    fixed : bool, optional
        True for distance based bandwidth and False for adaptive (nearest neighbor) bandwidth, by default False
    kernel : str, optional
        type of kernel function used to weight observations, by default "bisquare"
    n_jobs : int, optional
        The number of jobs to run in parallel. ``-1`` means using all processors by default ``-1``
    fit_global_model : bool, optional
        Determines if the global baseline model shall be fitted alognside the geographically weighted.
    **kwargs
        Additional keyword arguments passed to ``model`` initialisation
    """

    def __init__(
        self,
        model,
        fixed: bool = False,
        kernel: str | Callable = "bisquare",
        n_jobs: int = -1,
        search_method: str = "golden_section",
        criterion: str = "aic",
        min_bandwidth: int | float | None = None,
        max_bandwidth: int | float | None = None,
        interval: int | float | None = None,
        max_iterations: int = 100,
        tolerance: float = 1e-2,
        verbose: bool = False,
        **kwargs,
    ) -> None:
        self.model = model
        self.kernel = kernel
        self.fixed = fixed
        self.model_kwargs = kwargs
        self.n_jobs = n_jobs
        self.search_method = search_method
        self.criterion = criterion
        self.min_bandwidth = min_bandwidth
        self.max_bandwidth = max_bandwidth
        self.interval = interval
        self.max_iterations = max_iterations
        self.tolerance = tolerance
        self.verbose = verbose

    def fit(self, X: pd.DataFrame, y: pd.Series, geometry: gpd.GeoSeries) -> None:
        if self.search_method == "interval":
            self._interval(X=X, y=y, geometry=geometry)
        elif self.search_method == "golden_section":
            self._golden_section(X=X, y=y, geometry=geometry, tolerance=self.tolerance)

        self.optimal_bandwidth = self.oob_scores.idxmin()

        return self

    def _score(
        self, X: pd.DataFrame, y: pd.Series, geometry: gpd.GeoSeries, bw: int | float
    ) -> float:
        """Fit the model ans report criterion score.

        In case of invariant y in a local model, returns -np.inf
        """
        try:
            rf = GWM(
                model=self.model,
                bandwidth=bw,
                fixed=self.fixed,
                kernel=self.kernel,
                n_jobs=self.n_jobs,
                fit_global_model=False,
                measure_performance=False,
                strict=True,
                **self.model_kwargs,
            ).fit(X=X, y=y, geometry=geometry)
            log_likelihood = -metrics.log_loss(y, rf.focal_proba_)
            n, k = X.shape
            match self.criterion:
                case "aic":
                    return self._aic(k, n, log_likelihood)
                case "bic":
                    return self._bic(k, n, log_likelihood)
                case "aicc":
                    return self._aicc(k, n, log_likelihood)

        except ValueError:  # invariant subset
            return np.inf

    def _interval(self, X: pd.DataFrame, y: pd.Series, geometry: gpd.GeoSeries) -> None:
        """Fit models using the equal interval search.

        Parameters
        ----------
        X : pd.DataFrame
            Independent variables
        y : pd.Series
            Dependent variable
        geometry : gpd.GeoSeries
            Geographic location
        """
        oob_scores = {}
        bw = self.min_bandwidth
        while bw <= self.max_bandwidth:
            self.oob_scores[bw] = self._score(X=X, y=y, geometry=geometry, bw=bw)
            if self.verbose:
                print(f"Bandwidth: {bw:.2f}, score: {self.oob_scores[bw]:.2f}")
            bw += self.interval
        self.oob_scores = pd.Series(oob_scores, name="oob_score")

    def _golden_section(
        self, X: pd.DataFrame, y: pd.Series, geometry: gpd.GeoSeries, tolerance: float
    ) -> None:
        delta = 0.38197
        if self.fixed:
            pairwise_distance = pdist(geometry.get_coordinates())
            min_dist = np.min(pairwise_distance)
            max_dist = np.max(pairwise_distance)

            a = min_dist / 2.0
            c = max_dist * 2.0
        else:
            a = 40 + 2 * X.shape[1]
            c = len(geometry)

        if self.min_bandwidth:
            a = self.min_bandwidth
        if self.max_bandwidth:
            c = self.max_bandwidth

        b = a + delta * np.abs(c - a)
        d = c - delta * np.abs(c - a)

        diff = 1.0e9
        iters = 0
        oob_scores = {}
        while diff > tolerance and iters < self.max_iterations and a != np.inf:
            if not self.fixed:  # ensure we use int as possible bandwidth
                b = int(b)
                d = int(d)

            if b in oob_scores:
                score_b = oob_scores[b]
            else:
                score_b = self._score(X=X, y=y, geometry=geometry, bw=b)
                if self.verbose:
                    print(
                        f"Bandwidth: {f'{b:.2f}'.rstrip('0').rstrip('.')}, score: {score_b:.2f}"
                    )
                oob_scores[b] = score_b

            if d in oob_scores:
                score_d = oob_scores[d]
            else:
                score_d = self._score(X=X, y=y, geometry=geometry, bw=d)
                if self.verbose:
                    print(
                        f"Bandwidth: {f'{d:.2f}'.rstrip('0').rstrip('.')}, score: {score_d:.2f}"
                    )
                oob_scores[d] = score_d

            if score_b <= score_d:
                c = d
                d = b
                b = a + delta * np.abs(c - a)

            else:
                a = b
                b = d
                d = c - delta * np.abs(c - a)

            diff = np.abs(score_b - score_d)

        self.oob_scores = pd.Series(oob_scores, name="oob_score")

    def _aic(self, k, n, log_likelihood):
        return 2 * k - 2 * log_likelihood

    def _bic(self, k, n, log_likelihood):
        return -2 * log_likelihood + k * np.log(n)

    def _aicc(self, k, n, log_likelihood):
        return self._aic(k, n, log_likelihood) + 2 * k * (k + 1) / (n - k - 1)

Golden section search with a fixed distance bandwidth.

In [ ]:
search = BandwidthSearch(
    RandomForestClassifier,
    fixed=True,
    n_jobs=-1,
    search_method="golden_section",
    criterion="aic",
    max_iterations=10,
    min_bandwidth=250_000,
    max_bandwidth=2_000_000,
    verbose=True,
)
search.fit(
    gdf.iloc[:, 9:15],
    gdf["STATE_NAME"],
    gdf.geometry,
)

Get the optimal one.

In [ ]:
search.optimal_bandwidth

Golden section search with an adaptive KNN bandwidth.

In [ ]:
search = BandwidthSearch(
    LogisticRegression,
    fixed=False,
    n_jobs=-1,
    search_method="golden_section",
    criterion="aic",
    max_iterations=10,
    tolerance=0.1,
    verbose=True,
    max_iter=500,  # passed to log regr
)
search.fit(
    pd.DataFrame(
        preprocessing.scale(gdf.iloc[:, 9:15]), columns=gdf.iloc[:, 9:15].columns
    ),
    gdf["STATE_NAME"],
    gdf.geometry,
)

Get the optimal one.

In [ ]:
search.optimal_bandwidth